In [1]:
!pip install fvcore

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install av

In [4]:
import torch

In [5]:
pip install pytorchvideo

In [6]:
import json
import urllib
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [7]:
device = "GPU"

In [8]:
import pandas as pd
import cv2
import os

In [9]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 20
sampling_rate = 10
frames_per_second = 30

In [10]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size=(crop_size, crop_size))
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second
start_sec = 0
end_sec = start_sec + clip_duration

In [11]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, classes, transforms=None, max_frames= None):

        self.data_path = data_path
        self.classes = classes
        self.transforms = transforms
        self.allfiles = []
        for classes in os.listdir(data_path):
            if not os.path.isfile(classes):
                deep_dir = os.path.join(data_path,classes)
                for j in os.listdir(deep_dir):
                    self.allfiles.append(deep_dir+'/'+j)
        self.max_frames = max_frames

    def __len__(self):
        return len(self.allfiles)

    def read_video(self, path):

        video = EncodedVideo.from_path(path)

        # Load the desired clip
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

        # Apply a transform to normalize the video input
        video_data = self.transforms(video_data)
        # Move the inputs to the desired device
        inputs = video_data["video"]
        return inputs

    def __getitem__(self, idx) :
        v_path = self.allfiles[idx]
        label = v_path.split('/')[-2]
        return self.read_video(v_path), self.classes.index(label)

In [12]:
classes = pd.read_csv('classes.csv')
classes


,class_idx,class_name
0,0,cartwheel
1,1,catch
2,2,clap
3,3,climb
4,4,dive
5,5,draw_sword
6,6,dribble
7,7,fencing
8,8,flic_flac
9,9,golf


In [13]:
arra = []
for row in classes.iterrows():
    print(row[1][1])
    arra.append(row[1][1])

cartwheel
catch
clap
climb
dive
draw_sword
dribble
fencing
flic_flac
golf
handstand
hit
jump
pick
pour
pullup
push
pushup
shoot_ball
sit
situp
swing_baseball
sword_exercise
throw


In [ ]:
#!rm 'drive/MyDrive/hak/train_dataset_train/videos/.DS_Store'

In [14]:
DataSet = CustomDataset('drive/MyDrive/hak/train_dataset_train/videos',classes=arra, transforms = transform)

In [15]:
DataSet[0][1]

20

In [16]:
dataloaders = torch.utils.data.DataLoader(DataSet, batch_size=4,
                                              num_workers=2)

dataset_sizes = len(DataSet)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
len(dataloaders)

707

In [18]:
video_path = 'drive/MyDrive/hak/(HQ)_Rock_Climbing_-_Free_Solo_Speed_Climb_-_Dan_Osman_climb_f_cm_np1_fr_med_4.avi'

In [ ]:
video_path

'archery.mp4'

In [ ]:
# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class and load the video
video = EncodedVideo.from_path('drive/MyDrive/hak/7.avi')

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs = inputs.to(device)

In [ ]:
# Pass the input clip through the model
preds = model(inputs[None, ...])

# Get the predicted classes
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=5).indices[0]

# Map the predicted classes to the label names
pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
print("Top 5 predicted labels: %s" % ", ".join(pred_class_names))

Top 5 predicted labels: air drumming, singing, robot dancing, finger snapping, smoking


In [ ]:
torch.save(model, 'good_model.pt')

In [ ]:
model = torch.load('good_model.pt')
model.eval()

Training model


In [19]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

In [25]:
def train_model(model, criterion, optimizer, scheduler = None, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders:
                    inputs = inputs.to(device)
                    print(labels)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    print("step2")
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels)
                #if phase == 'train':
                #    print("step")
                #    scheduler.step()

                epoch_loss = running_loss / dataset_sizes
                epoch_acc = running_corrects.double() / dataset_sizes

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [26]:
model_ft = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)
# Set to GPU or CPU

model_ft = model_ft.to(device)
num_ftrs = model_ft.blocks[5].proj.in_features
model_ft.blocks[5].proj = nn.Linear(num_ftrs, 24)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [27]:
model_ft = train_model(model_ft, criterion, optimizer_ft,
                       num_epochs=25)

Epoch 0/24
----------
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 20, 20, 20])
step2
tensor([20, 18, 18, 18])
step2
tensor([18, 18, 18, 18])
step2
tensor([18, 18, 18, 18])
step2
tensor([18, 18, 18, 18])
step2
tensor([18, 18, 18, 18])
step2
tensor([18, 18, 1

RuntimeError: ignored

In [50]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
device.reset()

AttributeError: ignored

In [ ]:
for inputs, labels in dataloaders:
  print(inputs.shape)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])
torch.Size([4, 3, 20, 256, 256])


KeyboardInterrupt: ignored